In [2]:
# Setup movie data preprocessing environment
import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#from collecting_user_data import User

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD

In [8]:
# Ingest cleaned movie data
movies = pd.read_csv("../data/movies.csv")

# Ingest ratings csv file to gather user ratings of movies
ratings = pd.read_csv("../data/ratings_small.csv")

In [11]:
# Add rating_id to movies dataframe to look up ratings from other users when using collaborative filtering
id_map = pd.read_csv('../data/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].fillna(000000).astype(int)
id_map['tmdbId'] = id_map['tmdbId'].replace(000000, np.nan)
id_map.columns = ['rating_id', 'id']
movies = movies.merge(id_map, on="id")

In [18]:
display(movies)

,genres,id,overview,popularity,poster_path,release_date,runtime,title,vote_average,keywords,soup,rating_id
0,"['animation', 'comedy', 'family']",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",jealousy toy boy friendship friends rivalry bo...,1
1,"['adventure', 'fantasy', 'family']",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,104.0,Jumanji,6.9,"['boardgame', 'disappearance', ""basedonchildre...",boardgame disappearance basedonchildren'sbook ...,2
2,"['romance', 'comedy']",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,101.0,Grumpier Old Men,6.5,"['fishing', 'bestfriend', 'duringcreditsstinge...",fishing bestfriend duringcreditsstinger oldmen...,3
3,"['comedy', 'drama', 'romance']",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,127.0,Waiting to Exhale,6.1,"['basedonnovel', 'interracialrelationship', 's...",basedonnovel interracialrelationship singlemot...,4
4,['comedy'],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,106.0,Father of the Bride Part II,5.7,"['baby', 'midlifecrisis', 'confidence', 'aging...",baby midlifecrisis confidence aging daughter m...,5
...,...,...,...,...,...,...,...,...,...,...,...,...
7278,"['crime', 'drama', 'thriller', 'western']",338766,A divorced dad and his ex-con brother resort t...,12.565896,/6YOrNBdoXvT8aC5VPLkkN6t5z0V.jpg,2016-08-12,102.0,Hell or High Water,7.2,"['desperation', 'texas', 'bankrobber', 'brothe...",desperation texas bankrobber brother shooting ...,161582
7279,"['drama', 'thriller']",314420,A night out turns deadly when three girls brea...,6.236714,/dw0xE56whflv0OgfEw3lwdWxLRD.jpg,2015-01-25,75.0,Body,5.0,['mansion'],mansion drama thriller,161830
7280,"['comedy', 'horror', 'sciencefiction']",390989,The new installment of the Sharknado franchise...,4.574494,/jcP3HFXF1BIW9LmBrDusbbDZjBG.jpg,2016-07-31,85.0,Sharknado 4: The 4th Awakens,4.3,"['sharkattack', 'sequel', 'farce', 'lasvegas',...",sharkattack sequel farce lasvegas creaturefeat...,161918
7281,['drama'],159550,A man must cope with the loss of his wife and ...,0.038998,/yWp7PgydSlxlhl7benKhTnCvRjN.jpg,2001-09-23,85.0,The Last Brickmaker in America,7.0,"['friendship', 'brickmaking']",friendship brickmaking drama,161944


Train Collaborative Filtering Model

In [322]:
# Create copies of df's so we don't transform the df in unusual ways by accident
ratings_df = ratings.copy(deep=True)
movies_df = movies.copy(deep=True)

# Instantiate reader object and svd object
reader = Reader(rating_scale=(1, 5))
algo = SVD()

# Load data into surprise dataset format with reader
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

svd = SVD(n_epochs=5)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset by converting the CF dataset into a Surprice Trainset object
trainset = data.build_full_trainset()
svd.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9188  0.9171  0.9073  0.9157  0.9132  0.9144  0.0040  
MAE (testset)     0.7103  0.7094  0.7030  0.7096  0.7068  0.7078  0.0027  
Fit time          0.92    0.92    0.91    0.92    0.93    0.92    0.01    
Test time         0.16    0.07    0.16    0.07    0.16    0.13    0.04    


In [438]:
# Define function to apply to each row of a dataframe that would estimate the movie likeness for each user in question
def get_est(similar_users, movie, model):
    total_est = 0
    num_similar = len(similar_users)
    if not num_similar:
        return total_est
    
    for user in similar_users:
        est = model.predict(uid=user, iid=movie).est
        total_est += est
    
    return total_est / num_similar

Make Appropriate Recommendation

In [387]:
# Grabbing the inputs from the user
user = User()
titles = list(movies.title.values)
user.get_searchTerms()
user.get_fav_movie(titles)

In [388]:
print("User search terms: ", user.search_terms)
print("User favorite movie: ", user.fav_movie)

User search terms:  ['comedy romance', 'love action funny']
User favorite movie:  The Dark Knight


In [389]:
def top_movies_rec(movies = movies):
    ranked_titles = []
    ranked_movies = movies.copy(deep=True)
    ranked_movies = ranked_movies.sort_values("popularity", ascending=False)
    for idx in range(10):
        ranked_titles.append([movies.title.iloc[idx], movies.rating_id.iloc[idx]])
    return ranked_titles

In [390]:
def content_based_rec(movies = movies, search_terms = []):
    ranked_titles = []
    
    df = movies.copy(deep=True)
    
    # Calculate the average "vote_average" to limit the recs to movies that are above average
    rating_avg = round(df.vote_average.mean(), 0)
    df = df[df.vote_average >= rating_avg]
    
    # Creating a copy of the last row of the dataset, which we will use to input the user's input
    new_row = df.iloc[-1,:].copy()
    
    # Adding the input to the new row
    new_row.iloc[-2] = " ".join(search_terms)
  
    # Adding the new row to the dataset
    df = df.append(new_row)
  
    # Vectorizing the entire matrix
    count = CountVectorizer(analyzer = 'word', stop_words='english', ngram_range=(1, 2))
    count_matrix = count.fit_transform(df['soup'])

    # Pairwise cosine similarity and creating the cosine matrix
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
  
    # Sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Append top ranked df based on search terms to ranked_titles
    for i in range(1, 51):
        indx = sim_scores[i][0]
        ranked_titles.append([df['title'].iloc[indx], df['rating_id'].iloc[indx]])
        
    return ranked_titles

In [391]:
def collab_based_rec(movies = movies, ratings = ratings, fav_movie = None):
    df = movies.copy(deep=True)
    
    # Get fav_movie id for the User's favorite movie to analyize
    fav_movie_id = df[df.title == fav_movie]['rating_id'].values[0]
    
    # Get similiar users that have highly rated the users fav_movie in the past (top 15 users)
    similar_users = set(ratings[(ratings.movieId == fav_movie_id) & (ratings.rating >= 4)]["userId"])

    # Estimate the ratings the User would give each movie based on the similar users (predicted) ratings
    df['est'] = df.apply(lambda x: get_est(similar_users, x.id, svd), axis=1)

In [439]:
def hybrid_rec(movies = movies, search_terms = [], fav_movie = None):
    df = movies.copy(deep=True)
    
    # Get content-based recommendations
    content_results = content_based_rec(movies=movies, search_terms=search_terms)
    
    # Transform content based recs to a df for SVD estimations
    content_recs_df = pd.DataFrame(content_results).reset_index()
    content_recs_df.columns = ['content_rank', 'title', 'id']
    
    # Get fav_movie id for the User's favorite movie to analyize
    fav_movie_id = df[df.title == fav_movie]['rating_id'].values[0]
    
    # Get similiar users that have highly rated the users fav_movie in the past (top 15 users)
    similar_users = set(ratings[(ratings.movieId == fav_movie_id) & (ratings.rating >= 4)]["userId"])

    # Estimate the ratings the User would give each movie based on the similar users (predicted) ratings
    content_recs_df['est'] = content_recs_df.apply(lambda x: get_est(similar_users, x.id, svd), axis=1)
    
    # Sort by est and content_rank to re-rank the movies in the content-based list to be more personalized to the user
    content_recs_df = content_recs_df.sort_values(["est", "content_rank"], ascending=[False, True])
    
    # Collect the top 10 recommendations for the User
    ranked_titles = []
    for i in range(10):
        ranked_titles.append([content_recs_df['title'].iloc[i], content_recs_df['id'].iloc[i]])
        
    return content_recs_df

In [440]:
def make_recommendation(movies = movies, ratings = ratings, search_terms = [], fav_movie = None):
    if (not search_terms or search_terms == ['', '']) and not fav_movie:
        print("top_movie_rec")
        return top_movies_rec(movies)
    elif not fav_movie and search_terms:
        print("content_based_rec")
        return content_based_rec(movies=movies, search_terms=search_terms)
    elif fav_movie and (not search_terms or search_terms == ['', '']):
        print("collab_based_rec")
        return collab_based_rec(movies=movies, ratings=ratings, fav_movie=fav_movie)
    else:
        print("hybrid_rec")
        return hybrid_rec(movies=movies, search_terms=search_terms, fav_movie=fav_movie)

In [441]:
make_recommendation(movies=movies, ratings=ratings, search_terms=user.search_terms, fav_movie=user.fav_movie)

hybrid_rec


/var/folders/04/382s5ryn4cg90lmcyzj298g00000gn/T/ipykernel_24463/46158753.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row)


,content_rank,title,id,est
37,37,It Happened One Night,905,4.054306
29,29,The 40 Year Old Virgin,35836,3.830883
42,42,How to Marry a Millionaire,4357,3.733307
48,48,Safety Not Guaranteed,95088,3.725967
34,34,The Ugly Truth,70183,3.702448
18,18,Happy Accidents,4738,3.701136
45,45,How to Lose Friends & Alienate People,62113,3.695687
33,33,Pillow Talk,6183,3.693884
14,14,My Man Godfrey,947,3.693711
12,12,The Palm Beach Story,2937,3.691053
